# Gray-Scott Model for Coarse-Grained Reaction-Diffusion

In this Jupyter notebook, we will be simulating a Turing pattern generator, called the Gray-Scott Reaction-Diffusion model. Gray-Scott model is a model that has only two variables: a prey and a predator. Predator eats the prey to multiply and prey is present in the environment constantly.

**NOTE**: You will need to place this file on the same level as another folder named "*/gs_images*". ImageIO will not always create this folder automatically, so it may need to be created manually.

The Simulate function will take in the same parameters as the Diffuse function from the diffusion tutorial, but it will also take parameters f and k corresponding to the Gray-Scott feed and kill parameters, respectively. The simulation is in fact very similar to the diffusion notebook except for a very slight change that we make by adding the feed, kill, and predator-prey reactions when we update the matrices A and B containing the concentrations of the two particles over all the cells in the grid.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time
from scipy import signal
import imageio

%matplotlib inline

images = []
f_images = []

def simulate(
    numIter, 
    A, B, F, 
    rf, growth_a, growth_b, growth_f, 
    dt, dA, dB, dF, lapl, 
    plot_iter, filename):
    '''
    Simulate function
    Description: Simulate the Gray-Scott model for numIter iterations.
    Inputs:
        - numIter:  number of iterations
        - A:                prey matrix
        - B:                predator matrix
        - F:                fusion matrix
        - rf:               rate of fusion
        - growth_a:         growth rate of A
        - growth_b:         growth rate of B
        - growth_f:         growth of F
        - dt:               time constant
        - dA:               prey diffusion constant
        - dB:               predator diffusion constant
        - dF:               fusion diffusion constant
        - lapl:             3 x 3 Laplacian matrix to calculate diffusion

    Outputs:
        - A_matrices:   Prey matrices over the course of the simulation
        - B_matrices:   Predator matrices over the course of the simulation
    '''

    print("Running Simulation")
    start = time.time()

    # Run the simulation
    for iter in range(numIter):
        A_new = A + (dA * signal.convolve2d(A, lapl, mode='same', boundary='fill', fillvalue=0) + (growth_a * A) - (rf * A * B)) * dt
        B_new = B + (dB * signal.convolve2d(B, lapl, mode='same', boundary='fill', fillvalue=0) + (growth_b * B) - (rf * A * B)) * dt
        F_new = F + (dF * signal.convolve2d(F, lapl, mode='same', boundary='fill', fillvalue=0) + (growth_f * F) + (rf * A * B)) * dt
        A = np.copy(A_new)
        B = np.copy(B_new)
        F = np.copy(F_new)

        A = A / A.max()
        B = B / B.max()
        F = F / F.max()

        if (iter % plot_iter is 0):
            plt.clf()
            fig, axs = plt.subplots(2)
            axs[0].imshow((B / (A+B)), cmap='Spectral')
            axs[0].set_title('B')
            axs[0].axis('off')
            axs[1].imshow(F / (A+B), cmap='Spectral')
            axs[1].set_title('F')
            axs[1].axis('off')
            name = filename+str(iter)+'.png'
            plt.savefig(name)
            images.append(imageio.imread(name))
            plt.close()
            
            # plt.clf()
            # plt.imshow((F / (A+B)),cmap='Spectral')
            # plt.axis('off')
            # f_name = filename+str(iter)+'_fusion.png'
            # plt.savefig(f_name)
            # f_images.append(imageio.imread(f_name))

    print('Simulation time: ', time.time() - start)
    
    return A, B

In [ ]:
# _*_*_*_*_*_*_*_*_* GRID PROPERTIES *_*_*_*_*_*_*_*_*_*
grid_size = 101  # Needs to be odd
numIter = 5000
seed_size = 21  # Needs to be an odd number
A = np.ones((grid_size, grid_size))
B = np.zeros((grid_size, grid_size))
F = np.zeros((grid_size, grid_size))

# Seed the predators
mid = int(grid_size / 2)
seed_radius = int(seed_size / 2)
B[mid - seed_radius:mid + seed_radius + 1,
  mid - seed_radius:mid + seed_radius + 1] = np.ones((seed_size, seed_size))


In [ ]:
# _*_*_*_*_*_*_*_*_* SIMULATION VARIABLES *_*_*_*_*_*_*_*_*_*

rf = 0.1 # Rate of fusion

growth_a = 0.5
growth_b = 1
growth_f = 0.75

dt = 1.0
dA = 0.5
dB = 2.0
dF = 0.5

lapl = np.array([[0.05, 0.2, 0.05], [0.2, -1.0, 0.2], [0.05, 0.2, 0.05]])
plot_iter = 50

filename = 'gs_images/f55_k117'

'''
simulate(
    numIter, 
    A, B, F, 
    rf, growth_a, growth_b, growth_f, 
    dt, dA, dB, dF, lapl, 
    plot_iter, filename)
'''

simulate(numIter, A, B, F, rf, 
         growth_a, growth_b, growth_f, 
         dt, dA, dB, dF, lapl, plot_iter, filename)
imageio.mimsave(filename + '.gif', images)
